ABECTO does not longer use Jupyter Notebooks, due to a major architecture change. The latest version using Jupyter Notebooks is available on DOI:[10.5281/zenodo.4304533](https://doi.org/10.5281/zenodo.4304533) ([open on mybinder.org](https://mybinder.org/v2/zenodo/10.5281/zenodo.4304533/?filepath=abecto-tutorial.ipynb)). The most recent version of ABECTO is referred by DOI:[10.5281/zenodo.3786194](https://doi.org/10.5281/zenodo.3786194).

*This note remains to preserve a link in DOI:[10.1007/978-3-030-62327-2_24](https://doi.org/10.1007/978-3-030-62327-2_24).*